In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/09 14:42:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
ref_parent_file = "../../data/parquets/w1118.parquet"
alt_parent_file = "../../data/parquets/oregonr.parquet"
sample_file_glob = "../../data/parquets/WT-G0-*.parquet"
ref_parent_name = "w1118"
alt_parent_name = "oregonr"

ref_qual_cutoff = 200
ref_depth_cutoff = 30

ref_out_file = "../../data/parquets/reference.parquet"
sample_out_file = "../../data/parquets/progeny.parquet"

In [3]:
df = spark.read.parquet(ref_parent_file, alt_parent_file)
dist_df = sc.parallelize(df)
df.createOrReplaceTempView("df")

In [4]:
parents = spark.sql(
    "SELECT sample, chromosome, CAST(position AS INTEGER) AS int_pos, quality, genotype, depth, allele_depth, reference, variant, (CASE WHEN variant='.' THEN reference ELSE variant END) AS mod_variant FROM df"
)
parents.createOrReplaceTempView("parents")

In [5]:
check_unique_variants = spark.sql(
    """SELECT chromosome, int_pos, COUNT(variant) AS n_variants FROM parents
        GROUP BY chromosome, int_pos
        HAVING n_variants >= 2"""
)
check_unique_variants.createOrReplaceTempView("check_unique_variants")

In [6]:
temp_ref_query = """SELECT
    sample,
    parents.chromosome,
    parents.int_pos,
    reference,
    mod_variant,
    quality,
    genotype,
    depth,
    allele_depth
    FROM parents
    INNER JOIN check_unique_variants ON (
        parents.chromosome = check_unique_variants.chromosome
        AND parents.int_pos = check_unique_variants.int_pos
        )
    WHERE genotype!='0/1'
        AND quality > {ref_qual_cutoff}
        AND LENGTH(mod_variant) <= 1
        AND LENGTH(reference) <= 1
        AND depth > {ref_depth_cutoff}"""

temp_ref = spark.sql(
    temp_ref_query,
    ref_qual_cutoff=ref_qual_cutoff,
    ref_depth_cutoff=ref_depth_cutoff
)
temp_ref.createOrReplaceTempView("temp_ref")

In [15]:
parent_query = """SELECT
    sample,
    chromosome,
    int_pos,
    mod_variant
    FROM temp_ref
    WHERE sample={parent_name}"""

ref_parent = spark.sql(
    parent_query,
    parent_name=ref_parent_name
)
ref_parent = ref_parent.withColumnRenamed("mod_variant", "ref_allele")
ref_parent.createOrReplaceTempView("ref_parent")

In [17]:
alt_parent = spark.sql(
    parent_query,
    parent_name=alt_parent_name
)
alt_parent = alt_parent.withColumnRenamed("mod_variant", "alt_parent")
alt_parent.createOrReplaceTempView("alt_parent")

In [19]:
ref = spark.sql(
    """SELECT * FROM ref_parent
        FULL JOIN alt_parent ON (
            ref_parent.chromosome = alt_parent.chromosome
            AND ref_parent.int_pos = alt_parent.int_pos
            )
        ORDER BY ref_parent.chromosome, ref_parent.int_pos"""
)
ref.createOrReplaceTempView("ref")

ref.show()

24/08/09 14:27:33 ERROR Executor: Exception in task 4.0 in stage 1.0 (TID 5)8]8]
java.lang.OutOfMemoryError: Java heap space
24/08/09 14:27:33 ERROR Executor: Exception in task 16.0 in stage 1.0 (TID 17)
java.lang.OutOfMemoryError: Java heap space
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:102)
	at org.apache.spark.io.LZ4CompressionCodec.compressedOutputStream(CompressionCodec.scala:151)
	at org.apache.spark.serializer.SerializerManager.wrapForCompression(SerializerManager.scala:159)
	at org.apache.spark.serializer.SerializerManager.wrapStream(SerializerManager.scala:134)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:165)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:308)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
ref_out = spark.sql(
    """SELECT chromosome, int_pos AS position, ref_allele AS reference, alt_allele AS variant FROM ref""",
    ref=ref
)
ref_out.write.parquet(ref_out_file)